# Table of contents
1. [Preprocess country names](#matching_countries)
    1. [Plots Af](#plots_af)
2. [Preprocess of countries in the gdp and pop cases](#gdp-pop_matching)
    2. [Plots Af gdp and population](#gdp_plots)

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
plt.rcParams['figure.dpi']=100

%config InlineBackend.figure_format = 'retina'

In [3]:
gpd.__version__

'0.8.1'

## Preprocess country names <a name="matching_countries"></a>

In [97]:
df = pd.read_csv("normalization/datasets_intersection/af_food_trade_mean.csv") 
df=df.rename(columns={'Unnamed: 0':'name'})

world = gpd.read_file('world_shapefiles/Longitude_Graticules_and_World_Countries_Boundaries-shp/99bfd9e7-bb42-4728-87b5-07f8c8ac631c2020328-1-1vef4ev.lu5nk.shp')

col1=df.iloc[:,0]
col2=world.iloc[:,1]

def intersection_countries(col1,col2,prin=False):
    
    """ Compares two columns of string values based on string values of each column
    and returns a dictionary whose keys and values are the old and new names respectively.
    """
    
    cont=set(col1).intersection(col2)
    comparar1=set(col1)-set(cont)
    
    comparar2=set(col2)-set(col1)
    listadf_arenombrar = list(comparar1)
    listacomparar = list(comparar2)
    
    oldnew={}
    for country in listacomparar:
        for extract_count in listadf_arenombrar:
            if extract_count.find(country) != -1 or country.find(extract_count) !=-1:
                if prin:
                    print(extract_count, '------',country)
                oldnew[extract_count] = country
    return oldnew
    
oldnew=intersection_countries(col1,col2)

In [99]:
df['name']=df['name'].replace(oldnew)

In [100]:
cont=set(df.name).intersection(world.CNTRY_NAME)
dfleftover=set(df.name)-set(cont)
wleftover=set(world.CNTRY_NAME.unique())-set(df.name.unique())
print(len(dfleftover))

30


In [101]:
np.sort(list(dfleftover))

array(['Belarus', 'Belgium-Luxembourg', 'Cabo Verde',
       'China, Hong Kong SAR', 'China, Macao SAR', "Cote d'Ivoire",
       'Czechoslovakia', "Democratic People's Republic of Korea",
       'Eswatini', 'Ethiopia PDR', 'Falkland Islands (Malvinas)',
       'Holy See', 'Johnston Island', 'Korea, Rep.', 'Kyrgyz Republic',
       'Lao PDR', 'Micronesia, Fed. Sts.', 'Palestine', 'Réunion',
       'Saint Helena, Ascension and Tristan da Cunha',
       'Saint Pierre and Miquelon', 'Serbia and Montenegro',
       'Slovak Republic', 'South Sudan', 'Sudan (former)', 'Timor-Leste',
       'US Minor Is.', 'USSR', 'Unspecified Area', 'Yugoslav SFR'],
      dtype='<U44')

In [102]:
np.sort(list(wleftover))

array(['Antarctica', 'Baker Island', 'Byelarus', 'Cape Verde',
       'Falkland Islands (Islas Malvinas)',
       'Federated States of Micronesia',
       'French Southern & Antarctic Lands', 'Gaza Strip',
       'Glorioso Islands', 'Guernsey', 'Heard Island & McDonald Islands',
       'Howland Island', 'Ivory Coast', 'Jan Mayen', 'Jarvis Island',
       'Jersey', 'Johnston Atoll', 'Juan De Nova Island', 'Kyrgyzstan',
       'Laos', 'Macau', 'Man, Isle of', 'Norfolk Island', 'North Korea',
       'Northern Mariana Islands', 'Paracel Islands', 'Reunion',
       'Slovakia', 'South Georgia and the South Sandwich Is',
       'South Korea', 'Spratly Islands', 'St. Helena',
       'St. Pierre and Miquelon', 'Svalbard', 'Swaziland', 'Wake Island',
       'West Bank', 'Zaire'], dtype='<U39')

### cherry-picking countries 

In [103]:
oldnew={'Belarus':'Byelarus','Cabo Verde':'Cape Verde','Cote d\'Ivoir':'Ivory Coast',\
        'Democratic People\'s Republic of Korea':'North Korea','Eswatini':'Swaziland',\
        'Falkland Islands (Malvinas)':'Falkland Islands (Islas Malvinas)','Johnston Island':'Johnston Atoll',\
       'Korea, Rep.':'South Korea','Kyrgyz Republic':'Kyrgyzstan','Lao PDR':'Laos',\
        'Micronesia, Fed. Sts.':'Federated States of Micronesia',\
        'Saint Pierre and Miquelon':'St. Pierre and Miquelon','Slovak Republic':'Slovakia',\
       }
df['name']=df['name'].replace(oldnew)

In [104]:
cont=set(df.name).intersection(world.CNTRY_NAME)
dfleftover=set(df.name)-set(cont)
wleftover=set(world.CNTRY_NAME.unique())-set(df.name.unique())
print(len(dfleftover))

18


In [130]:
# df.to_csv('choropleths_af_food_trade_mean.csv',index=False) 

## Choropleths of Antifragility without considering either GDP or Population <a name="plots_af"></a>

In [8]:
df = pd.read_csv('datasets_choroplets/choropleths_af_food_trade_mean.csv')

#Import shp file of the world and ignore Antarctica
world = gpd.read_file('world_shapefiles/Longitude_Graticules_and_World_Countries_Boundaries-shp/99bfd9e7-bb42-4728-87b5-07f8c8ac631c2020328-1-1vef4ev.lu5nk.shp')
world = world[world.CNTRY_NAME !='Antarctica']
df.rename(columns={'name':'CNTRY_NAME'},inplace=True)

world_af=pd.merge_ordered(world,df,on='CNTRY_NAME')

In [211]:
missing_kwds={"color": "lightgrey","edgecolor": "black","hatch": "///","label": "Missing values"}


for antifrag in world_af.columns[3:]:
    
    fig,ax = plt.subplots(figsize=(20,10))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right",size="3%",pad=0.1)
    
    vmin=world_af.iloc[:,3:].min(numeric_only=True).min()
    vmax=world_af.iloc[:,3:].max(numeric_only=True).max()
    
    
    world_af.plot(column=antifrag,ax=ax,legend=True,cax=cax,missing_kwds=missing_kwds,vmin=vmin,vmax=vmax)
    ax.set_title(antifrag,fontsize=30)
    
    savepath='choroplets_AF/nonormalized/'
    if not os.path.exists(savepath):
        os.makedirs(savepath)
    plt.savefig(savepath+antifrag+'.png')
    plt.close()

## Preprocess country names, gdp and population cases <a name="gdp-pop_matching"></a>

In [185]:
df = pd.read_csv('normalization/datasets_intersection/gdp_pop_af_food_trade_percountry.csv')
df=df.rename(columns={'Unnamed: 0':'name'})


#Import shp file of the world and ignore Antarctica
world = gpd.read_file('world_shapefiles/Longitude_Graticules_and_World_Countries_Boundaries-shp/99bfd9e7-bb42-4728-87b5-07f8c8ac631c2020328-1-1vef4ev.lu5nk.shp')
world = world[world.CNTRY_NAME !='Antarctica']

col1=df.iloc[:,0]
col2=world.iloc[:,1]

oldnew=intersection_countries(col1,col2)

In [186]:
df['name']=df['name'].replace(oldnew)
cont=set(df.name).intersection(world.CNTRY_NAME)
dfleftover=set(df.name)-set(cont)
wleftover=set(world.CNTRY_NAME.unique())-set(df.name.unique())
print(len(dfleftover))

20


In [134]:
oldnew={'Belarus':'Byelarus','Cabo Verde':'Cape Verde','Cote d\'Ivoir':'Ivory Coast',\
        'Democratic People\'s Republic of Korea':'North Korea','Eswatini':'Swaziland',\
        'Falkland Islands (Malvinas)':'Falkland Islands (Islas Malvinas)','Johnston Island':'Johnston Atoll',\
       'Korea, Rep.':'South Korea','Kyrgyz Republic':'Kyrgyzstan','Lao PDR':'Laos',\
        'Micronesia, Fed. Sts.':'Federated States of Micronesia',\
        'Saint Pierre and Miquelon':'St. Pierre and Miquelon','Slovak Republic':'Slovakia',\
       }

df['name']=df['name'].replace(oldnew)

In [135]:
df['name']=df['name'].replace(oldnew)
cont=set(df.name).intersection(world.CNTRY_NAME)
dfleftover=set(df.name)-set(cont)
wleftover=set(world.CNTRY_NAME.unique())-set(df.name.unique())
print(len(dfleftover))

12


In [137]:
# df.to_csv('datasets_choroplets/choroplets_gdp_pop_af_food_trade_percountry.csv',index=False) 

## Choropleths of Antifragility GDP and Population cases <a name="gdp_plots"></a>

In [4]:
df = pd.read_csv('datasets_choroplets/choroplets_gdp_pop_af_food_trade_percountry.csv')
df=df.rename(columns={'name':'CNTRY_NAME'})


#Import shp file of the world and ignore Antarctica
world = gpd.read_file('world_shapefiles/Longitude_Graticules_and_World_Countries_Boundaries-shp/99bfd9e7-bb42-4728-87b5-07f8c8ac631c2020328-1-1vef4ev.lu5nk.shp')
world = world[world.CNTRY_NAME !='Antarctica']

world_gpd_pop_af=pd.merge_ordered(world,df,on='CNTRY_NAME')

In [212]:
missing_kwds={"color": "lightgrey","edgecolor": "black","hatch": "///","label": "Missing values"}

for index,antifrag in enumerate(world_gpd_pop_af.columns[3:]):
    
    fig,ax = plt.subplots(figsize=(20,10))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right",size="3%",pad=0.1)
    
    vmin=world_af.iloc[:,3:].min(numeric_only=True).min()
    vmax=world_af.iloc[:,3:].max(numeric_only=True).max()
    
    
    world_gpd_pop_af.plot(column=antifrag,ax=ax,legend=True,cax=cax,missing_kwds=missing_kwds,vmin=vmin,vmax=vmax)
    ax.set_title(antifrag,fontsize=30)
    
    savepath='choroplets_AF/gdp_population/'
    if not os.path.exists(savepath):
        os.makedirs(savepath)
    plt.savefig(savepath+antifrag+'.png')
    plt.close()